In [1]:
!dir

 Volume in drive C is Acer
 Volume Serial Number is D812-1956

 Directory of C:\Code Playground\Palmprintdetection_project

24-01-2023  22:08    <DIR>          .
21-01-2023  21:30    <DIR>          ..
24-01-2023  10:20    <DIR>          .ipynb_checkpoints
14-11-2022  01:49           655,831 9939-Article Text-17682-1-10-20210714.pdf
23-01-2023  20:33    <DIR>          dataset
13-11-2022  22:04    <DIR>          palm detection
24-01-2023  22:08            10,415 PalmPrint Detection model.ipynb
24-01-2023  22:02           965,459 Preprocessing_dataset.ipynb
23-01-2023  10:28            99,412 testing with sample images.ipynb
24-01-2023  22:02        14,440,167 X_guass.pickle
24-01-2023  22:02        14,440,167 X_meanth.pickle
24-01-2023  22:02             1,595 y_guass.pickle
24-01-2023  22:02             1,595 y_meanth.pickle
               8 File(s)     30,614,641 bytes
               5 Dir(s)  262,896,103,424 bytes free


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import cv2 as cv
import re 
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import Dense,Activation,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [3]:
#Meanth data
pickle_in=open("X_meanth.pickle","rb")
X_meanth=pickle.load(pickle_in)
pickle_in.close()
pickle_in=open("y_meanth.pickle","rb")
y_meanth=pickle.load(pickle_in)
pickle_in.close()

In [4]:
X_meanth.shape

(361, 200, 200, 1)

In [5]:
y_meanth.shape

(361,)

In [6]:
len(X_meanth)

361

In [7]:
len(y_meanth)

361

In [8]:
Xmtrain,Xmtest,ymtrain,ymtest=train_test_split(X_meanth,y_meanth,test_size=0.2,random_state=42)

In [9]:
print(len(Xmtrain),len(Xmtest),len(ymtrain),len(ymtest))

288 73 288 73


In [10]:
type(Xmtrain)
type(ymtrain)
ymtrain.dtype

dtype('int32')

In [36]:
from keras.optimizers import Adam

model = Sequential()

# Add the first layer (convolutional layer)
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',input_shape=Xmtrain.shape[1:]))

# Add the second layer (max pooling layer)
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Add the third layer (convolutional layer)
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',input_shape=Xmtrain.shape[1:]))

# Add the forth layer (max pooling layer)
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Add the fifth layer (flatten layer)
model.add(Flatten())

# Add the output layer
model.add(Dense(12, activation='softmax'))

# Compile the model
# optimizer = Adam(learning_rate=0.00001)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(Xmtrain,ymtrain,epochs=4,validation_split=0.1)

Epoch 1/4
9/9 [==============================] - 5s 449ms/step - loss: 1699.2988 - accuracy: 0.1004 - val_loss: 407.7896 - val_accuracy: 0.1034
Epoch 2/4
9/9 [==============================] - 4s 433ms/step - loss: 120.5435 - accuracy: 0.1544 - val_loss: 4.3854 - val_accuracy: 0.2414
Epoch 3/4
9/9 [==============================] - 4s 432ms/step - loss: 0.4555 - accuracy: 0.8803 - val_loss: 2.0297 - val_accuracy: 0.4828
Epoch 4/4
9/9 [==============================] - 4s 436ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 2.3020 - val_accuracy: 0.5862


In [37]:
np.array(list(map(np.argmax,model.predict(Xmtest)))).reshape(-1)

3/3 [==============================] - 0s 79ms/step


array([ 0, 10,  7,  7,  5,  5,  5,  7,  9,  3,  1,  7,  7, 11, 11,  5,  4,
        9,  9, 10,  3, 10,  3,  7,  3,  3, 11, 10,  3,  3,  4,  8,  1,  3,
        3,  3,  5,  0,  5,  5,  5,  5,  7,  7, 10,  0,  5,  8,  3,  5,  3,
        7,  8, 11,  3,  4,  6,  2,  5,  7,  5,  3,  9,  3, 10,  6,  3,  3,
        7,  7,  6, 10,  7], dtype=int64)

In [38]:
ymtest

array([ 2, 10,  7,  7,  6,  4,  4,  7,  9,  1,  1,  0,  7,  5,  1,  6,  2,
        9,  9,  9,  2,  9,  0,  7,  3,  4, 11, 10,  0,  7,  4,  9,  5,  4,
        3,  1,  2,  0,  1,  1,  4,  5,  7,  7, 10,  3,  2,  8,  0,  1,  8,
        7,  8, 11, 10,  4,  6,  2,  6,  3,  3,  4,  9, 11, 10,  6,  3,  9,
        7,  7,  6, 11,  7])

In [39]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 200, 200, 32)      320       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 100, 100, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 50, 50, 64)       0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 160000)            0         
                                                                 
 dense_10 (Dense)            (None, 12)              

In [47]:
# from sklearn.metrics import r2_score
# r2_score(ymtest,model.predict(Xmtest))
len(Xmtest)

73

In [48]:
len(ymtest)

73

In [40]:
model.save("Palmprint_detection_meanth.model")

INFO:tensorflow:Assets written to: Palmprint_detection_meanth.model\assets


INFO:tensorflow:Assets written to: Palmprint_detection_meanth.model\assets


In [41]:
pickle_in=open("X_guass.pickle","rb")
X_guass=pickle.load(pickle_in)
pickle_in.close()
pickle_in=open("y_guass.pickle","rb")
y_guass=pickle.load(pickle_in)
pickle_in.close()

In [42]:
X_guass.shape

(361, 200, 200, 1)

In [43]:
y_guass.shape

(361,)

In [44]:
X_guass

array([[[[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [  0],
         [255],
         [255]],

        ...,

        [[255],
         [255],
         [  0],
         ...,
         [  0],
         [255],
         [255]],

        [[255],
         [255],
         [  0],
         ...,
         [  0],
         [255],
         [255]],

        [[  0],
         [255],
         [255],
         ...,
         [  0],
         [255],
         [255]]],


       [[[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [  0],
         ...,
         [255],
         [255],
         [255]],

        [[  0],
         [255],
         [255],
         ...,
         [255],
         [

In [45]:
y_guass

array([ 1,  2,  8,  2,  1,  5,  3,  7,  1,  7, 10,  0,  4, 10,  4,  1,  3,
        8, 11,  0,  3,  6,  7,  9,  3,  6,  1,  6, 11,  3,  5,  9,  5,  3,
        7,  9,  9,  2,  2,  4,  3,  1,  3,  8,  5,  3,  9,  5, 11,  4,  2,
        9,  5,  4,  6,  2,  3,  6,  7,  2,  0,  9,  0,  0, 11,  7,  0,  1,
        9,  2, 10,  8,  9,  3,  4,  2,  3, 10, 10,  7,  8, 11,  6,  9,  2,
       10,  7,  1,  4, 11,  9,  1,  4,  1,  1,  2,  7,  2,  8,  9,  2,  3,
        0,  0,  9,  5, 10,  5,  8,  4,  0, 10,  5,  4, 10,  3,  9,  3,  4,
       10,  3,  6,  9,  5,  2, 11,  1,  9,  4,  2,  2,  2,  0,  3,  5,  0,
        4, 11,  6,  3, 10,  0,  1,  7,  7,  9,  8,  0,  1,  0, 11, 11,  9,
        3, 10,  6,  8,  0,  0,  9,  5, 11,  6,  0,  8,  6,  5,  3,  8, 10,
        7,  3, 10,  2,  2,  7,  2,  9,  0,  1,  1,  1,  0, 11,  1, 10,  9,
        7,  7,  7, 11,  4,  0,  5,  5, 11,  6,  8,  3,  7,  4,  4,  0,  1,
       11, 11,  4,  3, 10,  4,  1,  3,  5,  2,  5,  0,  8,  1,  9,  5,  6,
        1,  2,  6,  7,  2

In [51]:
Xgtrain,Xgtest,ygtrain,ygtest=train_test_split(X_guass,y_guass,test_size=0.2,random_state=42)

In [52]:
print(len(Xgtrain),len(Xgtest),len(ygtrain),len(ygtest))

288 73 288 73


In [63]:
from keras.optimizers import Adam

modelGuass = Sequential()

# Add the first layer (convolutional layer)
modelGuass.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',input_shape=Xgtrain.shape[1:]))

# Add the second layer (max pooling layer)
modelGuass.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Add the third layer (convolutional layer)
modelGuass.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',input_shape=Xgtrain.shape[1:]))

# Add the forth layer (max pooling layer)
modelGuass.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Add the fifth layer (flatten layer)
modelGuass.add(Flatten())

# Add the output layer
modelGuass.add(Dense(12, activation='softmax'))

# Compile the model
# optimizer = Adam(learning_rate=0.00001)
modelGuass.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


modelGuass.fit(Xgtrain,ygtrain,epochs=4,validation_split=0.1)

Epoch 1/4
9/9 [==============================] - 5s 493ms/step - loss: 1369.9785 - accuracy: 0.0618 - val_loss: 104.9225 - val_accuracy: 0.0690
Epoch 2/4
9/9 [==============================] - 4s 454ms/step - loss: 24.2639 - accuracy: 0.1274 - val_loss: 3.3628 - val_accuracy: 0.0000e+00
Epoch 3/4
9/9 [==============================] - 4s 440ms/step - loss: 1.8503 - accuracy: 0.5212 - val_loss: 2.0207 - val_accuracy: 0.3793
Epoch 4/4
9/9 [==============================] - 4s 443ms/step - loss: 0.2022 - accuracy: 0.9923 - val_loss: 2.2681 - val_accuracy: 0.5172


In [64]:
np.array(list(map(np.argmax,modelGuass.predict(Xgtest)))).reshape(-1)

3/3 [==============================] - 0s 78ms/step


array([11, 10, 11,  5, 11,  9,  3, 10, 11,  4, 11,  4,  5, 11, 11,  9,  9,
        7, 11,  2,  5,  8,  9,  6,  5,  5, 11,  8, 11,  5,  0,  9,  5,  3,
       10, 10,  0, 11,  2,  2,  7, 11,  9,  7,  4,  2,  5, 10,  5, 11,  4,
        2,  7, 11,  8, 11,  1,  2,  4, 11, 10, 11, 11,  9, 11,  2, 11,  2,
        8,  4, 11,  8, 11], dtype=int64)

In [65]:
ymtest

array([ 2, 10,  7,  7,  6,  4,  4,  7,  9,  1,  1,  0,  7,  5,  1,  6,  2,
        9,  9,  9,  2,  9,  0,  7,  3,  4, 11, 10,  0,  7,  4,  9,  5,  4,
        3,  1,  2,  0,  1,  1,  4,  5,  7,  7, 10,  3,  2,  8,  0,  1,  8,
        7,  8, 11, 10,  4,  6,  2,  6,  3,  3,  4,  9, 11, 10,  6,  3,  9,
        7,  7,  6, 11,  7])

In [67]:
modelGuass.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 200, 200, 32)      320       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 100, 100, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 50, 50, 64)       0         
 g2D)                                                            
                                                                 
 flatten_16 (Flatten)        (None, 160000)            0         
                                                                 
 dense_16 (Dense)            (None, 12)              

In [68]:
modelGuass.save("Palmprint_detection_guass.model")

INFO:tensorflow:Assets written to: Palmprint_detection_guass.model\assets


INFO:tensorflow:Assets written to: Palmprint_detection_guass.model\assets
